In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr'

%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/mink/notebooks/CameraTraps')  # append this repo to PYTHONPATH
sys.path.append('/home/mink/lib/ai4eutils')

In [3]:
import json
import os
from collections import Counter, defaultdict
from random import sample
from shutil import copyfile
from multiprocessing.pool import ThreadPool

import numpy as np
import pandas as pd
from tqdm import tqdm

import path_utils  # ai4eutils

from data_management.megadb.schema import sequences_schema_check
from data_management.megadb.converters.cct_to_megadb import process_sequences

# fws_lehnen_mice

In [4]:
dataset_name = 'fws_lehnen_mice'

container_root = '/mink_disk_0/camtraps/fws-lehnen/'  
path_prefix = 'Jumping mice 2017 processed photos/'

path_to_output = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}.json' 
path_to_output_temp = f'/home/mink/camtraps/data/megadb_jsons/{dataset_name}_temp.json' 

## Step 0 - Add an entry to the `datasets` table

Done

## Step 1 - Prepare the `sequence` objects to insert into the database

Extracted location info from the file names, but no sequence.

In [5]:
folder = os.path.join(container_root, path_prefix)

paths = path_utils.recursive_file_list(folder)
len(paths)
paths = sorted([p.split(folder)[1] for p in paths if path_utils.is_image_file(p)])
len(paths)

63039

63038

In [6]:
paths[-100]
'_'.join(paths[-100].split('#')[1].split('_')[:6])

'BdA-#9_9_20_17_18A02_canal_00017_10-17-2017_18-43.JPG'

'9_9_20_17_18A02_canal'

In [7]:
locations = set()
sequences = []

for fn in paths:
    location = '_'.join(fn.split('#')[1].split('_')[:5])
    locations.add(location)
    sequences.append({
        'dataset': dataset_name,
        'seq_id': f'dummy_{len(sequences)}',
        'images': [
            {
                'file': fn,
                'frame_num': 1 # only one image, but easier for ingesting the annotations
            }
        ],
        'location': location,
        'class': ['__label_unavailable']
    })

In [8]:
len(locations)

80

In [9]:
len(sequences)
sequences[60038]

63038

{'dataset': 'fws_lehnen_mice',
 'seq_id': 'dummy_60038',
 'images': [{'file': 'BdA-#5_5_23_17_18A01_00314_6-17-2017_22-11.JPG',
   'frame_num': 1}],
 'location': '5_5_23_17_18A01',
 'class': ['__label_unavailable']}

## Step 2 - Pass the schema check

Once your metadata are in the MegaDB format for `sequence` items, we check that they conform to the format's schema.

If the format conforms, the following messages will be printed:

```
Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
```

For large datasets, the second step will take some time (~ a minute). 

Otherwise there will be an error message describing what's wrong. Please fix the issues until all checks are passed. You might need to write some snippets of code to loop through the `sequence` items to understand which entries have problems.

In [10]:
%%time

sequences_schema_check.sequences_schema_check(sequences)

Verified that the sequence items meet requirements not captured by the schema.
Verified that the sequence items conform to the schema.
CPU times: user 7.61 s, sys: 11.1 ms, total: 7.62 s
Wall time: 7.61 s


In [11]:
with open(path_to_output_temp, 'w', encoding='utf-8') as f:
    json.dump(sequences, f, indent=1, ensure_ascii=False)

### Step 2b - copy images to flat folder

In [12]:
def copy_file(src_path, dst_path):
    return copyfile(src_path, dst_path)

In [13]:
%%time

path_pairs = []
for seq in tqdm(sequences):
    seq_id = seq['seq_id']
    for im in seq['images']:
        src_path = os.path.join(container_root, path_prefix, im['file'])
        assert os.path.exists(src_path), src_path
        frame = 1
        dst_path = os.path.join('/mink_disk_0/camtraps/imerit12b', 
                                f'{dataset_name}.seq{seq_id}.frame{frame}.jpg')
        path_pairs.append((src_path, dst_path))

100%|██████████| 63038/63038 [00:06<00:00, 10295.11it/s]

CPU times: user 665 ms, sys: 453 ms, total: 1.12 s
Wall time: 6.13 s


In [14]:
len(path_pairs)  # non-empty images out of total of 19221 (38%)
path_pairs[-100]

63038

('/mink_disk_0/camtraps/fws-lehnen/Jumping mice 2017 processed photos/BdA-#9_9_20_17_18A02_canal_00017_10-17-2017_18-43.JPG',
 '/mink_disk_0/camtraps/imerit12b/fws_lehnen_mice.seqdummy_62938.frame1.jpg')

In [15]:
%%time

with ThreadPool(8) as pool:
    dst_paths = pool.starmap(copy_file, path_pairs)

CPU times: user 23 s, sys: 1min 1s, total: 1min 24s
Wall time: 3min 11s


In [16]:
len(dst_paths)

63038